## Download model and tokenizer
https://github.com/airsplay/lxmert/tree/master?tab=readme-ov-file#pre-trained-models

In [1]:
# setup environment 
!git clone https://github.com/microsoft/unilm.git
!cd unilm/beit3
!pip install -r requirements.txt   

!wget https://github.com/addf400/files/releases/download/beit3/beit3_base_patch16_480_vqa.pth  
!wget https://github.com/addf400/files/releases/download/beit3/beit3.spm  

--2025-11-20 18:14:10--  https://github.com/addf400/files/releases/download/beit3/beit3_base_patch16_480_vqa.pth
Resolving github.com (github.com)... 20.200.245.247
Connecting to github.com (github.com)|20.200.245.247|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/717694328/a382a6e7-7b1e-4d79-b9e6-702a6b1081f3?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-20T09%3A56%3A54Z&rscd=attachment%3B+filename%3Dbeit3_base_patch16_480_vqa.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-20T08%3A56%3A50Z&ske=2025-11-20T09%3A56%3A54Z&sks=b&skv=2018-11-09&sig=dTUpg43H9tyY6NYIfiONb4o9XiL3QvFWGj8RjaSbf3g%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MzYzMzY1MSwibmJmIjoxNzYzNjMwMDUxLCJwYXRoIjoicmVsZWF

## Get model 

In [9]:
from timm.models import create_model 
from transformers import XLMRobertaTokenizer

tokenizer = XLMRobertaTokenizer("./beit3.spm") 
model = "beit_base_patch16_224"
task = "vqav2" 
drop_path = 0.1 
vocab_size = 64010

if task in ("flickr30k", "coco_retrieval"): 
    model_config = "%s_retrieval" % model
elif task in ("coco_captioning", "nocaps"): 
    model_config = "%s_captioning" % model
elif task in ("imagenet"): 
    model_config = "%s_imageclassification" % model
else:
    model_config = "%s_%s" % (model, task)

model_config 

In [ ]:

model = create_model(
    model_config,
    pretrained=False,
    drop_path_rate=drop_path,
    vocab_size=vocab_size,
    checkpoint_activations=None, 
)
device = "cuda:1" 
model.to(device) 

## Dataset 

### custom dataset 

In [14]:
import sys 
sys.path.append('unilm/beit3')
from datasets import CustomDataset

CustomDataset.make_dataset_index(
    data_path="/home/work/yuna/HPA/data",
    tokenizer=tokenizer,
    json_data_path="/home/work/yuna/HPA/data/vqav2_val_1k",
)

FileNotFoundError: [Errno 2] No such file or directory: '/home/work/yuna/HPA/data/vqav2_val_1k/train.json'

## Evaluation

In [2]:
'''
이번 대회 데이터셋에 맞게, 아래 부분을 수정했습니다.(수정한 코드는 제 git에서 확인해주세요.)
    <수정 내용>
    dataset.py : CustomDataset Class 추가 및 task2dataset에 vqacustom 추가
    utils.py : import torch._six를 torch로 수정(module not found 에러 해결),
                pos_tokens = pos_tokens.float() 코드 추가(575번째 줄, 참고 : https://github.com/microsoft/unilm/issues/932),
                dump_prediction 함수(838번째 줄)에 있는 torch.distributed.barrier() 삭제(single GPU에서의 eval 위함, multi GPU의 경우 삭제 안해도 됨)
    engine_for_finetuning.py : def get_handler 함수에(442번째 줄) args.task == "vqacustom" 추가
    modeling_finetune.py : vqacustom 모델들 추가 및 num_classes부분 수정(label 수에 맞게)
    run_beit3_finetuning.py : parser.add_argument --task 부분에 vqacustom 추가, args.eval(357번째 줄)에 vqacustom 부분(367~370번째 줄) 추가
'''

'\n이번 대회 데이터셋에 맞게, 아래 부분을 수정했습니다.(수정한 코드는 제 git에서 확인해주세요.)\n    <수정 내용>\n    dataset.py : CustomDataset Class 추가 및 task2dataset에 vqacustom 추가\n    utils.py : import torch._six를 torch로 수정(module not found 에러 해결),\n                pos_tokens = pos_tokens.float() 코드 추가(575번째 줄, 참고 : https://github.com/microsoft/unilm/issues/932),\n                dump_prediction 함수(838번째 줄)에 있는 torch.distributed.barrier() 삭제(single GPU에서의 eval 위함, multi GPU의 경우 삭제 안해도 됨)\n    engine_for_finetuning.py : def get_handler 함수에(442번째 줄) args.task == "vqacustom" 추가\n    modeling_finetune.py : vqacustom 모델들 추가 및 num_classes부분 수정(label 수에 맞게)\n    run_beit3_finetuning.py : parser.add_argument --task 부분에 vqacustom 추가, args.eval(357번째 줄)에 vqacustom 부분(367~370번째 줄) 추가\n'

In [4]:
!python ./unilm/beit3/run_beit3_finetuning.py \
    --model beit3_base_patch16_384 \
    --task vqacustom \
    --batch_size 128 \
    --sentencepiece_model ./beit3.spm \
    --data_path /home/work/yuna/VLMEval/data/val2014 \
    --output_dir ./outputs/ \
    --eval
# --finetune /content/drive/MyDrive/Dacon_multimodal/finetuned_model_file/custom_model.pth \
# --input_size 224 \

Not using distributed mode
Namespace(aa='rand-m9-mstd0.5-inc1', auto_resume=True, batch_size=128, captioning_mask_prob=0.6, checkpoint_activations=None, clip_grad=None, color_jitter=0.4, crop_pct=None, cutmix=0, cutmix_minmax=None, data_path='/home/work/yuna/VLMEval/data/val2014', device='cuda', dist_eval=False, dist_on_itp=False, dist_url='env://', distributed=False, drop_path=0.1, drop_worst_after=12000, drop_worst_ratio=0.2, enable_deepspeed=False, epochs=20, eval=True, eval_batch_size=None, finetune='', initial_scale_power=16, input_size=224, label_smoothing=0.1, layer_decay=0.9, length_penalty=0.6, local_rank=-1, log_dir=None, lr=0.0005, min_lr=1e-06, mixup=0, mixup_mode='batch', mixup_prob=1.0, mixup_switch_prob=0.5, model='beit3_base_patch16_384', model_ema=False, model_ema_decay=0.9999, model_ema_force_cpu=False, model_key='model|module', model_prefix='', momentum=0.9, nb_classes=1000, num_beams=3, num_max_bpe_tokens=64, num_workers=0, opt='adamw', opt_betas=[0.9, 0.999], opt_e